In [1]:
import os
from EgoExoEMSDataset import *

OSError: /sfs/qumulo/qhome/cjh9fw/.local/lib/python3.8/site-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZN2at4_ops10zeros_like4callERKNS_6TensorEN3c108optionalINS5_10ScalarTypeEEENS6_INS5_6LayoutEEENS6_INS5_6DeviceEEENS6_IbEENS6_INS5_12MemoryFormatEEE

In [ ]:
root = "/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/May_2024/May24_updated_structure/ego_camera/ng1/1/"  # Folder in which all videos lie in a specific structure
annotation_file = "../../Annotations/main_annotation.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)

dataset = EgoExoEMSDataset(annotation_file=annotation_file,
                                video_base_path='',
                                fps=30, frames_per_clip=None, transform=transform)

# Access a sample
print(len(dataset))

# create a data loader
# batch size is 1 for simplicity and to ensure only a full clip related to a key step is given without collating.
# if batch size is greater than 1, collate_fn will be called to collate the data.
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)



124


In [ ]:
# Iterate over the data loader and print the shape of the batch
for batch in data_loader:
    print(batch['frames'].shape, batch['audio'].shape, batch['keystep_label'], batch['keystep_id'], batch['start_frame'], batch['end_frame'], batch['subject_id'], batch['trial_id'])
    # break   

torch.Size([1, 129, 3, 224, 224]) ['check_pulse_breathing'] tensor([3]) tensor([3377]) tensor([3506])
torch.Size([1, 65, 3, 224, 224]) ['place_bvm'] tensor([8]) tensor([3166]) tensor([3231])
torch.Size([1, 86, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([2780]) tensor([2866])
torch.Size([1, 32, 3, 224, 224]) ['no_action'] tensor([5]) tensor([1993]) tensor([2025])
torch.Size([1, 36, 3, 224, 224]) ['place_bvm'] tensor([8]) tensor([3188]) tensor([3224])
torch.Size([1, 237, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1174]) tensor([1411])
torch.Size([1, 207, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([4512]) tensor([4719])
torch.Size([1, 225, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1770]) tensor([1995])
torch.Size([1, 209, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1910]) tensor([2119])
torch.Size([1, 417, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([6202]) tensor([6619])
torch.Size([1, 131, 3, 224, 224]) ['check_pulse_br

[ WARN:0@896.335] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_FFMPEG_READ_ATTEMPTS (current value is 4096)


torch.Size([1, 15, 3, 224, 224]) ['check_responsiveness'] tensor([2]) tensor([0]) tensor([111])
torch.Size([1, 72, 3, 224, 224]) ['place_bvm'] tensor([8]) tensor([3630]) tensor([3702])
torch.Size([1, 424, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([3937]) tensor([4361])
torch.Size([1, 476, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1900]) tensor([2376])
torch.Size([1, 1378, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([225]) tensor([1603])
torch.Size([1, 103, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([2925]) tensor([3028])
torch.Size([1, 208, 3, 224, 224]) ['check_pulse_breathing'] tensor([3]) tensor([5005]) tensor([5213])
torch.Size([1, 91, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([1680]) tensor([1771])


[ WARN:0@995.312] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30026.765449 ms
[ WARN:0@995.312] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30026.828598 ms
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7e8ae80] stream 4, offset 0x315458cd: partial file


torch.Size([1, 29, 3, 224, 224]) ['check_responsiveness'] tensor([2]) tensor([1]) tensor([69])
torch.Size([1, 44, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([2559]) tensor([2603])
torch.Size([1, 250, 3, 224, 224]) ['administer_shock_aed'] tensor([7]) tensor([796]) tensor([1046])
torch.Size([1, 89, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([1279]) tensor([1368])
torch.Size([1, 457, 3, 224, 224]) ['administer_shock_aed'] tensor([7]) tensor([574]) tensor([1031])
torch.Size([1, 477, 3, 224, 224]) ['administer_shock_aed'] tensor([7]) tensor([1992]) tensor([2469])
torch.Size([1, 91, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([1837]) tensor([1928])
torch.Size([1, 542, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([2647]) tensor([3189])
torch.Size([1, 93, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([637]) tensor([730])
torch.Size([1, 201, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1367]) tensor([1568])
torch.Size([1, 459, 3, 224, 224]) 

...

torch.Size([1, 15, 3, 224, 224]) ['approach_patient'] tensor([1]) tensor([0]) tensor([87])
torch.Size([1, 87, 3, 224, 224]) ['place_bvm'] tensor([8]) tensor([551]) tensor([638])
torch.Size([1, 410, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([3363]) tensor([3773])


[ WARN:0@1145.780] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_FFMPEG_READ_ATTEMPTS (current value is 4096)


torch.Size([1, 15, 3, 224, 224]) ['check_pulse_breathing'] tensor([3]) tensor([0]) tensor([64])
torch.Size([1, 239, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1433]) tensor([1672])
torch.Size([1, 43, 3, 224, 224]) ['check_pulse_breathing'] tensor([3]) tensor([5312]) tensor([5355])
torch.Size([1, 88, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([1671]) tensor([1759])
torch.Size([1, 87, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([1567]) tensor([1654])
torch.Size([1, 133, 3, 224, 224]) ['check_pulse_breathing'] tensor([3]) tensor([6070]) tensor([6203])
torch.Size([1, 492, 3, 224, 224]) ['administer_shock_aed'] tensor([7]) tensor([4821]) tensor([5313])
torch.Size([1, 213, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([3504]) tensor([3717])
torch.Size([1, 112, 3, 224, 224]) ['check_pulse_breathing'] tensor([3]) tensor([68]) tensor([180])
torch.Size([1, 205, 3, 224, 224]) ['attach_defib_pads'] tensor([6]) tensor([198]) tensor([403])
torch.Size([1, 138, 3, 

[ WARN:0@1276.376] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_FFMPEG_READ_ATTEMPTS (current value is 4096)


torch.Size([1, 168, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([31]) tensor([199])
torch.Size([1, 120, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([3692]) tensor([3812])
torch.Size([1, 264, 3, 224, 224]) ['administer_shock_aed'] tensor([7]) tensor([2382]) tensor([2646])
torch.Size([1, 163, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([4405]) tensor([4568])
torch.Size([1, 30, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([1620]) tensor([1650])
torch.Size([1, 252, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([2049]) tensor([2301])
torch.Size([1, 200, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([4208]) tensor([4408])
torch.Size([1, 127, 3, 224, 224]) ['chest_compressions'] tensor([4]) tensor([2053]) tensor([2180])
torch.Size([1, 145, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([3223]) tensor([3368])
torch.Size([1, 104, 3, 224, 224]) ['compress_bvm'] tensor([9]) tensor([4718]) tensor([4822])
torch.Size([1, 89, 3, 224, 224